In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asus4\\OneDrive\\Documents\\DEEP LEARNING PROJECT\\DL-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus4\\OneDrive\\Documents\\DEEP LEARNING PROJECT\\DL-Project'

In [5]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass 
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from src.cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.constants import *

In [7]:
class ConfigManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories(
            [
                Path(config.tensorboard_log_dir),
                Path(model_ckpt_dir)
            ])
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_log_dir=Path(config.tensorboard_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

In [8]:
import os 
import urllib.request as request
import zipfile
import tensorflow as tf
import time

In [ ]:
class prepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tensorboard_callback(self):
        tensorboard_log_dir = os.path.join(
            self.config.tensorboard_log_dir,
            f"tb_logs_at_{time.strftime('%Y-%m-%d-%H-%M-%S')}"
        )
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir=tensorboard_log_dir
        )
        return tensorboard_callback 
     
    @property
    def _create_model_checkpoint_callback(self):
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        return model_checkpoint_callback
         
    @property
    def get_callbacks(self):
        return [
            self._create_tensorboard_callback,
            self._create_model_checkpoint_callback 
        ]

In [12]:
try:
    config_manager = ConfigManager()
    prepare_callbacks_config = config_manager.get_prepare_callbacks_config()

    prepare_callbacks = prepareCallbacks(config=prepare_callbacks_config)
    callbacks = prepare_callbacks.get_callbacks
except Exception as e:
    raise e

[2025-10-23 18:07:42,887]:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-10-23 18:07:42,891]:INFO:common:yaml file: params.yaml loaded successfully]
[2025-10-23 18:07:42,894]:INFO:common:created directory at : artifacts]
[2025-10-23 18:07:42,897]:INFO:common:created directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-10-23 18:07:42,899]:INFO:common:created directory at : artifacts\prepare_callbacks\checkpoint_dir]


In [14]:
import time 
timestramp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestramp}"

'tb_logs_at_2025-10-23-18-09-28'